In [97]:
"""
Algorithm in order to move a group of units in a strategy game:

Here we consider tanks and factories (inspired by https://www.youtube.com/watch?v=8kBQMQyLHME):
- each turn, you can move all your tanks
- every (N / number of factories) turns, you can pop a new tank at one of your factories
- the goal is to eliminate the adversary (no tank, no factories left)
"""

from collections import defaultdict, deque
from dataclasses import dataclass
import enum
import numpy as np
from typing import *

import imageio
from PIL import Image

In [117]:
"""
Game mechanics
"""


Position = Tuple[int, int]


class Move(enum.Enum):
    NONE = 0
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4
    
    
class Player(enum.Enum):
    PLAYER_1 = 0
    PLAYER_2 = 1
    
    def next_player(self):
        return self.PLAYER_1 if self == self.PLAYER_2 else self.PLAYER_2


@dataclass(frozen=True)
class Territory:
    width: int
    height: int
        
    def displace(self, position: Position, move: Move):
        x, y = position
        if move == Move.LEFT:
            x -= 1
        elif move == Move.RIGHT:
            x += 1
        elif move == Move.UP:
            y += 1
        elif move == Move.DOWN:
            y -= 1
        return (x, y) if 0 <= x < self.width and 0 <= y < self.height else position

    
class TankPositions:
    def __init__(self):
        self.positions = defaultdict(int)
    
    def add(self, position: Position):
        self.positions[position] += 1
    
    def remove(self, position: Position):
        count = self.positions.get(position, 0)
        if count == 0:
            raise "No tank at position " + str(position)
        elif count == 1:
            del self.positions[position]
        else:
            self.positions[position] = count - 1
    
    def __contains__(self, position):
        return position in self.positions
    
    def __repr__(self):
        return repr(self.positions)
    
    def items(self):
        return self.positions.items()


@dataclass(frozen=False)
class PlayerPieces:
    factories: Set[Position]
    tanks: TankPositions
        
    @classmethod
    def init_pieces(cls, territory: Territory, player: Player, nb_factories: int):
        factories = set()
        tanks = TankPositions()
        for y in np.random.choice(range(territory.height), size=nb_factories, replace=False):
            factories.add((cls.start_factory_column(territory, player), y))
            tanks.add((cls.start_tank_column(territory, player), y))
        return cls(tanks=tanks, factories=factories)
    
    @staticmethod
    def start_factory_column(territory: Territory, player: Player):
        return 0 if player == Player.PLAYER_1 else territory.width-1
    
    @staticmethod
    def start_tank_column(territory: Territory, player: Player):
        return 1 if player == Player.PLAYER_1 else territory.width-2

        
@dataclass(frozen=False)
class GameState:
    territory: Territory
    players: Dict[Player, PlayerPieces]
    
    @classmethod
    def init_game(cls, width: int, height: int, nb_factories: int):
        territory = Territory(width, height)
        return GameState(
            territory = territory,
            players = {
                Player.PLAYER_1: PlayerPieces.init_pieces(territory, Player.PLAYER_1, nb_factories),
                Player.PLAYER_2: PlayerPieces.init_pieces(territory, Player.PLAYER_2, nb_factories)
            })


@dataclass(frozen=False)
class Turn:
    tank_moves: List[Tuple[Position, Move]]
    pop_location: Position
    
    
class TankWars:
    # TODO - manage winner + punish mistakes by "you loose"
    # TODO - could represent the map of tanks as just a single map with 1 for player 1, -1 for player 2
    
    def __init__(self, game_state: GameState):
        self.current_player = Player.PLAYER_1
        self.game_state = game_state
    
    def get_territory(self) -> Territory:
        return self.game_state.territory
    
    def current_player_status(self) -> PlayerPieces:
        return self.game_state.players[self.current_player]
    
    def other_player_status(self) -> PlayerPieces:
        return self.game_state.players[self.current_player.next_player()]
    
    def play(self, turn: Turn):
        player = self.current_player_status()
        other_player = self.other_player_status()
        for curr_pos, move in turn.tank_moves:
            if curr_pos not in player.tanks:
                raise "Invalid move: tank does not exist"
            next_pos = self.game_state.territory.displace(curr_pos, move)
            if next_pos in other_player.tanks:
                other_player.tanks.remove(next_pos)
            else:
                player.tanks.add(next_pos)
            player.tanks.remove(curr_pos)
        if turn.pop_location in player.factories:
            player.tanks.add(turn.pop_location)
        else:
            raise "Invalid pop location"
        self.current_player = self.current_player.next_player()


"""
Diffusion algorithm based AI
"""


def neighbor_positions(territory: Territory, position: Position):
    x, y = position
    if x > 0:
        yield x-1, y
    if x < territory.width - 1:
        yield x+1, y
    if y > 0:
        yield x, y-1
    if y < territory.height - 1:
        yield x, y+1
        

def diffusion_map(territory: Territory, positions: List[Tuple[Position, float]], alpha: float):
    m = np.zeros((territory.width, territory.height))
    for (x, y), value in positions:
        m[x][y] = value
    to_visit = deque(positions)
    while to_visit:
        position, value = to_visit.popleft()
        for x, y in neighbor_positions(territory, position):
            if m[x][y] == 0:
                m[x][y] = value * alpha
                to_visit.append(((x, y), value * alpha))
    return m
                

def diffusion_ia_play(game: TankWars):
    territory = game.get_territory()
    player = game.current_player_status()
    other = game.other_player_status()
    
    enemy_tanks = diffusion_map(territory, [(tank, count * 10.) for tank, count in other.tanks.items()], alpha=0.8)
    enemy_factories = diffusion_map(territory, [(factory, 25.) for factory in other.factories], alpha=0.8)
    
    print(enemy_tanks)
    print(enemy_factories)
        
    

In [118]:
game_state = GameState.init_game(width=5, height=5, nb_factories=1)
game = TankWars(game_state)
game.current_player_status()

PlayerPieces(factories={(0, 1)}, tanks=defaultdict(<class 'int'>, {(1, 1): 1}))

In [119]:
'''
game.play(Turn(
    tank_moves=[((1, 7), Move.RIGHT)],
    pop_location=(0, 6)
))
game.game_state
'''

diffusion_ia_play(game)

[[ 2.097152  2.62144   3.2768    4.096     5.12    ]
 [ 2.62144   3.2768    4.096     5.12      6.4     ]
 [ 3.2768    4.096     5.12      6.4       8.      ]
 [ 4.096     5.12      6.4       8.       10.      ]
 [ 3.2768    4.096     5.12      6.4       8.      ]]
[[ 4.194304  5.24288   6.5536    8.192    10.24    ]
 [ 5.24288   6.5536    8.192    10.24     12.8     ]
 [ 6.5536    8.192    10.24     12.8      16.      ]
 [ 8.192    10.24     12.8      16.       20.      ]
 [10.24     12.8      16.       20.       25.      ]]


In [ ]:
# Main problem with the diffusion algorithm is that several tanks stacked will always behave the same (unless you tank into account each tank move)... and that tank will always move !
# TODO - you can also introduce a bit of random (ponderated move by the map around the tank)